In [5]:
# Import the required libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

# Read the apple.csv file from the Resources folder into a Pandas DataFrame
# Set the date column as the DataTimeIndex
aapl_df = pd.read_csv(
    Path("../../Class_Activities/M5_Class_2_Activities/Class_2_Activities/03-Inst_Intro_MonteCarlo/Resources/AAPL.csv"),
    index_col="date",
    parse_dates=True,
    infer_datetime_format=True)

# Review the DataFrame
aapl_df.head()

,close,volume,open,high,low
date,,,,,
2009-08-03,23.7757,98285279,23.6014,23.8057,23.5529
2009-08-04,23.6500,98772690,23.5614,23.6529,23.4586
2009-08-05,23.5871,105581942,23.6786,23.9128,23.4586
2009-08-06,23.4157,85271156,23.6543,23.7871,23.2986
2009-08-07,23.6443,96727219,23.6414,23.8000,23.5429


In [6]:
# Filter the date index and close columns
signals_df = aapl_df.loc[:,["close"]]

# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0

# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)

# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Review the DataFrame
signals_df.tail(10)

,close,SMA50,SMA100,Signal,Entry/Exit
date,,,,,
2019-07-19,202.59,193.3102,193.1920,1.0,0.0
2019-07-22,207.22,193.4402,193.5155,0.0,-1.0
2019-07-23,208.84,193.6734,193.8724,0.0,0.0
2019-07-24,208.67,194.1324,194.2094,0.0,0.0
2019-07-25,207.02,194.4996,194.5211,0.0,0.0
2019-07-26,207.74,194.8360,194.8432,0.0,0.0
2019-07-29,209.68,195.2280,195.1948,1.0,1.0
2019-07-30,208.78,195.6236,195.5576,1.0,0.0
2019-07-31,213.04,196.2226,195.9589,1.0,0.0


In [7]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="Apple - SMA50, SMA100, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)

In [8]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

# Buy a 500 share position when the dual moving average crossover Signal equals 1 (SMA50 is greater than SMA100)
# Sell a 500 share position when the dual moving average crossover Signal equals 0 (SMA50 is less than SMA100)
signals_df['Position'] = share_size * signals_df['Signal']

In [9]:
# Determine the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

In [10]:
# Multiply the close price by the number of shares held, or the Position
signals_df['Portfolio Holdings'] = signals_df['close'] * signals_df['Position']

In [11]:
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
signals_df['Portfolio Cash'] = initial_capital - (signals_df['close'] * signals_df['Entry/Exit Position']).cumsum()

In [12]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

In [13]:
# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

In [14]:
# Calculate the portfolio cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

In [15]:
# Print the DataFrame
signals_df.loc["2015-04-21":"2015-04-27"]

,close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
2015-04-21,126.91,126.5278,119.6350,1.0,0.0,500.0,0.0,63455.0,77752.9,141207.9,-0.002437,0.412079
2015-04-22,128.62,126.7058,119.7452,1.0,0.0,500.0,0.0,64310.0,77752.9,142062.9,0.006055,0.420629
2015-04-23,129.67,126.8588,119.8519,1.0,0.0,500.0,0.0,64835.0,77752.9,142587.9,0.003696,0.425879
2015-04-24,130.28,126.9668,119.9654,1.0,0.0,500.0,0.0,65140.0,77752.9,142892.9,0.002139,0.428929
2015-04-27,132.65,127.0906,120.1412,1.0,0.0,500.0,0.0,66325.0,77752.9,144077.9,0.008293,0.440779


In [16]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="Apple Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)

In [17]:
# 14.2.6  Risk/Reward Evaluation Metrics
signals_df.tail()

,close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
2019-07-26,207.74,194.8360,194.8432,0.0,0.0,0.0,0.0,0.0,168317.9,168317.9,0.000000,0.683179
2019-07-29,209.68,195.2280,195.1948,1.0,1.0,500.0,500.0,104840.0,63477.9,168317.9,0.000000,0.683179
2019-07-30,208.78,195.6236,195.5576,1.0,0.0,500.0,0.0,104390.0,63477.9,167867.9,-0.002674,0.678679
2019-07-31,213.04,196.2226,195.9589,1.0,0.0,500.0,0.0,106520.0,63477.9,169997.9,0.012689,0.699979
2019-08-01,208.43,196.6592,196.2542,1.0,0.0,500.0,0.0,104215.0,63477.9,167692.9,-0.013559,0.676929


In [18]:
# Create a list for the column name
columns = ['Backtest']

# Create a list holding the names of the new evaluation metrics
metrics = [
    'Annualized Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

# Initialize the DataFrame with index set to the evaluation metrics and the column
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

# Review the DataFrame
portfolio_evaluation_df

,Backtest
Annualized Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [19]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annualized Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)
portfolio_evaluation_df

,Backtest
Annualized Return,0.055245
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [20]:
# We already calculated the cumulative returns metric during our original backtesting process.
#   Use the last value in the “Portfolio Cumulative Returns” column of our signals_df

# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]
portfolio_evaluation_df

,Backtest
Annualized Return,0.055245
Cumulative Returns,0.676929
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [21]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annualized Return,0.055245
Cumulative Returns,0.676929
Annual Volatility,0.082953
Sharpe Ratio,NaN
Sortino Ratio,NaN


In [22]:
# Calculate Sharpe ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)
portfolio_evaluation_df

,Backtest
Annualized Return,0.055245
Cumulative Returns,0.676929
Annual Volatility,0.082953
Sharpe Ratio,0.665983
Sortino Ratio,NaN


In [23]:
# Calculate downside return values

# Create a DataFrame that contains the Portfolio Daily Returns column
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()

# Create a column to hold downside return values
sortino_ratio_df.loc[:,'Downside Returns'] = 0

# Find Portfolio Daily Returns values less than 0,
# square those values, and add them to the Downside Returns column
sortino_ratio_df.loc[sortino_ratio_df['Portfolio Daily Returns'] < 0,
                     'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2

# Review the DataFrame
sortino_ratio_df.tail()

,Portfolio Daily Returns,Downside Returns
date,,
2019-07-26,0.000000,0.000000
2019-07-29,0.000000,0.000000
2019-07-30,-0.002674,0.000007
2019-07-31,0.012689,0.000000
2019-08-01,-0.013559,0.000184


In [24]:
# Calculate the Sortino ratio

# Calculate the annualized return value
annualized_return = (
    sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate the annualized downside standard deviation value
downside_standard_deviation = (
    np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
)

# The Sortino ratio is reached by dividing the annualized return value
# by the downside standard deviation value
sortino_ratio = annualized_return/downside_standard_deviation

# Add the Sortino ratio to the evaluation DataFrame
portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df

,Backtest
Annualized Return,0.055245
Cumulative Returns,0.676929
Annual Volatility,0.082953
Sharpe Ratio,0.665983
Sortino Ratio,0.963309


In [25]:
# In general, a higher Sharpe ratio indicates a better risk/reward profile.
# We commonly get a Sharpe ratio of about 1.00 for a portfolio with a favorable risk-adjusted profile. 
# As with the Sharpe ratio, a higher Sortino ratio is better. 
# Annualized volatility: Spread around the Annualzed return: 0.055245 +- 0.082953

In [26]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock',
        'Entry Date',
        'Exit Date',
        'Shares',
        'Entry Share Price',
        'Exit Share Price',
        'Entry Portfolio Holding',
        'Exit Portfolio Holding',
        'Profit/Loss']
)

In [27]:
# Initialize iterative variables
entry_date = ""
exit_date = ""
entry_portfolio_holding = 0.0
exit_portfolio_holding = 0.0
share_size = 0
entry_share_price = 0.0
exit_share_price = 0.0

In [28]:
# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = row['Portfolio Holdings']
        share_size = row['Entry/Exit Position']
        entry_share_price = row['close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['close'] * row['Entry/Exit Position'])
        exit_share_price = row['close']
        profit_loss = exit_portfolio_holding - entry_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': 'AAPL',
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,AAPL,2009-12-22,2010-08-31,500.0,28.6228,34.7286,14311.40,17364.30,3052.90
1,AAPL,2010-09-21,2011-05-18,500.0,40.5386,48.5528,20269.30,24276.40,4007.10
2,AAPL,2011-07-26,2012-07-06,500.0,57.6300,86.5543,28815.00,43277.15,14462.15
3,AAPL,2012-08-07,2012-11-21,500.0,88.7020,80.2428,44351.00,40121.40,-4229.60
4,AAPL,2013-07-09,2013-07-16,500.0,60.3357,61.4564,30167.85,30728.20,560.35
5,AAPL,2013-08-15,2014-03-10,500.0,71.1300,75.8457,35565.00,37922.85,2357.85
6,AAPL,2014-05-06,2015-08-03,500.0,84.9157,118.4400,42457.85,59220.00,16762.15
7,AAPL,2015-12-09,2016-01-13,500.0,115.6200,97.3900,57810.00,48695.00,-9115.00
8,AAPL,2016-04-25,2016-06-14,500.0,105.0800,97.4600,52540.00,48730.00,-3810.00
9,AAPL,2016-08-22,2018-03-26,500.0,108.5100,172.7700,54255.00,86385.00,32130.00
